### Question 1

Install those libraries not preinstalled in Colab.

In [ ]:
! pip install -q 'lalsuite==6.66' 'bilby==0.6.3' 'gwpy==1.0.1'

     |████████████████████████████████| 28.5MB 155kB/s 
     |████████████████████████████████| 1.5MB 31.4MB/s 
     |████████████████████████████████| 1.4MB 53.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 1.6MB 46.5MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 2.6MB 47.9MB/s 


Import necessary libraries.

In [ ]:
from gwosc.datasets import event_gps, find_datasets
import numpy as np
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries
import bilby
from bilby.core.prior import Uniform, Cosine, Sine, Constraint, PowerLaw
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters

Let's list out all the events of 2019.

In [ ]:
t1 = 1230336018 # 2019-01-01 00:00:00
t2 = 1261872017 # 2019-12-31 23:59:59
events = find_datasets(type="event", detector='H1', segment=(t1, t2))
print(events)

['GW190408_181802-v1', 'GW190412-v1', 'GW190412-v2', 'GW190412-v3', 'GW190413_052954-v1', 'GW190413_134308-v1', 'GW190421_213856-v1', 'GW190426_152155-v1', 'GW190503_185404-v1', 'GW190512_180714-v1', 'GW190513_205428-v1', 'GW190514_065416-v1', 'GW190517_055101-v1', 'GW190519_153544-v1', 'GW190521-v1', 'GW190521-v2', 'GW190521-v3', 'GW190521_074359-v1', 'GW190527_092055-v1', 'GW190602_175927-v1', 'GW190701_203306-v1', 'GW190706_222641-v1', 'GW190707_093326-v1', 'GW190719_215514-v1', 'GW190720_000836-v1', 'GW190727_060333-v1', 'GW190728_064510-v1', 'GW190731_140936-v1', 'GW190803_022701-v1', 'GW190814-v1', 'GW190814-v2', 'GW190828_063405-v1', 'GW190828_065509-v1', 'GW190909_114149-v1', 'GW190915_235702-v1', 'GW190924_021846-v1', 'GW190929_012149-v1', 'GW190930_133541-v1']


Let's pick the first event, and define some variables for time duration for which we'll use the signal data.

In [ ]:
gps = event_gps(events[0])
ifo_list = bilby.gw.detector.InterferometerList([])
t1, t2 = gps - 2, gps + 2
psd_t1, psd_t2 = t1 - 128, t1
psd_alpha = 0.2

For both the Hanford and Livingston detectors, let's fetch open signal and psd data from LIGO archives.

In [ ]:
for det in ['H1', 'L1']:
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    data = TimeSeries.fetch_open_data(det, t1, t2)
    ifo.strain_data.set_from_gwpy_timeseries(data)

    psd_data = TimeSeries.fetch_open_data(det, psd_t1, psd_t2)
    psd = psd_data.psd(
        fftlength=4,
        overlap=0,
        window=("tukey", psd_alpha),
        method="median"
    )
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, 
        psd_array=psd.value)
    ifo_list.append(ifo)

/usr/local/lib/python3.6/dist-packages/gwpy/types/series.py:926: UserWarning: TimeSeries.crop given end larger than current end, crop will end when the Series actually ends.
  % type(self).__name__)


Make an outdir for bilby.

In [ ]:
bilby.core.utils.check_directory_exists_and_if_not_mkdir('outdir')
ifo_list.plot_data(outdir='outdir', label=events[0])

11:54 bilby INFO    : Generating frequency domain strain from given time domain strain.
11:54 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2
11:54 bilby INFO    : Generating frequency domain strain from given time domain strain.
11:54 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2


Define some priors, and create a waveform. Then we define a likelihood function and just run the sampler.

In [ ]:
priors = dict()
priors['a_1'] = Uniform(minimum=0, maximum=0.99, name='a_1', latex_label='$a_1$', unit=None, boundary='reflective')
priors['a_2'] = Uniform(minimum=0, maximum=0.99, name='a_2', latex_label='$a_2$', unit=None, boundary='reflective')
priors['chirp_mass'] = Uniform(minimum=25, maximum=31, name='chirp_mass', latex_label='$\\mathcal{M}$', unit='$M_{\\odot}$', boundary='reflective')
priors['dec'] = Cosine(name='dec', latex_label='$\\mathrm{DEC}$', unit=None, minimum=-1.5707963267948966, maximum=1.5707963267948966, boundary='reflective')
priors['luminosity_distance'] = PowerLaw(alpha=2, minimum=50, maximum=2000, name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None)
priors['mass_1'] = Constraint(minimum=10, maximum=80, name='mass_1', latex_label='$m_1$', unit=None)
priors['mass_2'] = Constraint(minimum=10, maximum=80, name='mass_2', latex_label='$m_2$', unit=None)
priors['mass_ratio'] = Uniform(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary='reflective')
priors['phase'] = Uniform(minimum=0, maximum=6.283185307179586, name='phase', latex_label='$\\phi$', unit=None, boundary='periodic')
priors['phi_12'] = Uniform(minimum=0, maximum=6.283185307179586, name='phi_12', latex_label='$\\Delta\\phi$', unit=None, boundary='periodic')
priors['phi_jl'] = Uniform(minimum=0, maximum=6.283185307179586, name='phi_jl', latex_label='$\\phi_{JL}$', unit=None, boundary='periodic')
priors['psi'] = Uniform(minimum=0, maximum=3.141592653589793, name='psi', latex_label='$\\psi$', unit=None, boundary='periodic')
priors['ra'] = Uniform(minimum=0, maximum=6.283185307179586, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic')
priors['theta_jn'] = Sine(name='theta_jn', latex_label='$\\theta_{JN}$', unit=None, minimum=0, maximum=3.141592653589793, boundary='reflective')
priors['tilt_1'] = Sine(name='tilt_1', latex_label='$\\theta_1$', unit=None, minimum=0, maximum=3.141592653589793, boundary='reflective')
priors['tilt_2'] = Sine(name='tilt_2', latex_label='$\\theta_2$', unit=None, minimum=0, maximum=3.141592653589793, boundary='reflective')

waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={'waveform_approximant': 'IMRPhenomPv2',
                        'reference_frequency': 50})

likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list, waveform_generator, priors=priors, time_marginalization=True,
    phase_marginalization=True, distance_marginalization=True)

result = bilby.run_sampler(
    likelihood, priors, sampler='dynesty', outdir='outdir', label=events[0],
    nlive=1000, walks=100, n_check_point=10000, check_point_plot=True,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters)

result.plot_corner()

12:55 bilby WARNING : The waveform_generator start_time is not equal to that of the provided interferometers. Overwriting the waveform_generator.
12:55 bilby WARNING : Prior not provided for geocent_time, using the BBH default.
12:55 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
12:55 bilby INFO    : Building lookup table for distance marginalisation.
12:59 bilby INFO    : Running for label 'GW190408_181802-v1', output will be saved to 'outdir'
12:59 bilby INFO    : Using LAL version Branch: None;Tag: lalsuite-v6.66;Id: 04a60e3ac9b6ecb285cf96d1137c6a62a3d5cfde;;Builder: Unknown User <>;Repository status: UNCLEAN: Modified working tree
12:59 bilby INFO    : Search parameters:
12:59 bilby INFO    :   a_1 = Uniform(minimum=0, maximum=0.99, name='a_1', latex_label='$a_1$', unit=None, boundary='reflective')
12:59 bilby INFO    :   a_2 = Uniform(minimum=0, maximum=0.99, name='a_2', latex_label='$a_2$', unit=None, boundary='reflective')
12:59 

0it [00:00, ?it/s]

12:59 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 600, 'nlive': 1000, 'first_update': None, 'walks': 100, 'npdim': None, 'rstate': None, 'queue_size': None, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 600, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7f00dbba5748>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5}
12:59 bilby INFO    : Checkpoint every n_check_point = 10000
12:59 bilby INFO    : Using dynesty version 1.0.1
12:59 bilby INFO    : Using the bilby-implemented rwalk sampl

3027it [03:47,  3.49it/s, bound:0 nc:19 ncall:21004 eff:14.4% logz-ratio=40.90+/-0.10 dlogz:46.023>0.10]

13:03 bilby INFO    : Writing checkpoint file outdir/GW190408_181802-v1_resume.pickle


...


7321it [5:37:41,  5.33s/it, bound:1860 nc:494 ncall:1894184 eff:0.4% logz-ratio=71.08+/-0.12 dlogz:13.099>0.10]

18:37 bilby INFO    : Writing checkpoint file outdir/GW190408_181802-v1_resume.pickle



7720it [6:13:34,  5.48s/it, bound:2060 nc:498 ncall:2094275 eff:0.4% logz-ratio=72.19+/-0.12 dlogz:11.589>0.10]

19:13 bilby INFO    : Writing checkpoint file outdir/GW190408_181802-v1_resume.pickle



8024it [6:41:08,  5.36s/it, bound:2213 nc:498 ncall:2246675 eff:0.4% logz-ratio=72.94+/-0.12 dlogz:10.539>0.10]